In [ ]:
import pandas as pd
import statsmodels.formula.api as sm
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
samples = pd.read_table('raw/lcm_samplekey.txt', index_col=['lcm_sampleID'])
samples.head()

In [ ]:
data = pd.read_table('raw/lcm.run2_data.txt').dropna(subset=['compound']).set_index(['Data_Filename', 'compound'])
data.head()

In [ ]:
calibration_data = (data.join(samples, on='Sample_ID')
                        [lambda x: x.concentration_mM.notnull()]
                        .reset_index())
butyrate_calibration_data = calibration_data[calibration_data['compound'] == 'butyrate']
butyrate_calibration_data = \
butyrate_calibration_data[['Data_Filename', 'Sample_ID',
                           'Area', 'Height',
                           'injection', 'concentration_mM']]
butyrate_calibration_data

In [ ]:
fit1 = sm.ols('concentration_mM ~ Area + 0', data=butyrate_calibration_data).fit()
fit1.summary()

In [ ]:
fit2 = sm.wls('concentration_mM ~ Area + 0',
              data=butyrate_calibration_data,
              weights=butyrate_calibration_data.concentration_mM ** (-2)).fit()
fit2.summary()

In [ ]:
(fit1.rsquared, fit2.rsquared)

In [ ]:
plt.scatter('Area', 'concentration_mM', data=butyrate_calibration_data)
plt.xscale('log')
plt.yscale('log')

xs = np.linspace(butyrate_calibration_data.Area.min(), butyrate_calibration_data.Area.max())
xs
plt.plot(xs, xs * fit1.params['Area'])
plt.plot(xs, xs * fit2.params['Area'])

In [ ]:
plt.scatter(fit2.predict(), fit2.resid_pearson)

In [ ]:
butyrate_data = data.xs('butyrate', level='compound')
butyrate_data['calc_conc'] = butyrate_data.Area * fit2.params['Area']
butyrate_data.head()

In [ ]:
butyrate_data.join(samples, on='Sample_ID').columns

In [ ]:
result

In [ ]:
result = (butyrate_data.join(samples, on='Sample_ID')
                       [['run', 'Sample_ID',
                         'sample_type', 'Ret._Time',
                         'Area', 'Height', 'fecal_weight',
                         'PBS_added', 'dilution',
                         'concentration_mM', 'calc_conc']]
                        [lambda x: x.sample_type == 'unknown'])
result.calc_conc.fillna(0, inplace=True)
result['sample_concentration'] = result.calc_conc / result.dilution
result.sample_concentration.plot.hist(bins=100)